## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
import keras
import numpy.matlib
import mat73
import colorcet as cc

from numpy import asarray
from scipy import stats, signal, io
from scipy.ndimage import median_filter as medfilt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.utils.extmath import randomized_svd as rsvd

from utils import tic, toc, hankel_matrix, scaler, legendre_polys, find_opt_lag
from models import linear_regression, VAE

%matplotlib notebook

2025-01-02 20:04:17.400653: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-02 20:04:17.537336: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-02 20:04:18.030395: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-01-02 20:04:18.030468: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

## Load and preprocess data

In [2]:
def load_pupil(subj,filt=True):
    data_dict = mat73.loadmat(data_dir + '/' + str(subj) + '_pupil.mat')
    pupil = data_dict['pupil'].reshape(-1,1)
    pupil = medfilt(pupil,21)
    
    if filt:
        Fs = 20
        dt = 1/Fs
        sos = signal.butter(1, [.01,.2],btype = 'bandpass', output = 'sos', fs = Fs)
        pupil = signal.sosfiltfilt(sos, pupil, axis=0)
    
    pupil = pupil[1000:-800,:]
    
    pupil = StandardScaler(with_std=False).fit_transform(pupil)

    return pupil

In [3]:
def load_brain(subj,modality='jrgeco',filt=True):
    data_dict = mat73.loadmat(data_dir + '/' + str(subj) + '_' + modality + '.mat')
    brain = data_dict['data'][:,mask_long]

    if filt:
        Fs = 20
        dt = 1/Fs
        sos = signal.butter(1, [.01,.2],btype = 'bandpass', output = 'sos', fs = Fs)
        brain = signal.sosfiltfilt(sos, brain, axis=0)
        
    brain = brain[1000:-800,:]
    
    brain = StandardScaler(with_std=False).fit_transform(brain)

    return brain

In [4]:
data_dir = '/home/ryanr/widefield'
outdir = '/home/ryanr/Allen/arousal_final/model_predictions'
mask = io.loadmat(data_dir+'/newmask.mat')
mask = mask["newmask"]
mask.dtype = bool
mask_long = mask.reshape(16384,order='F')
allen_ccf = mat73.loadmat(data_dir + '/allen_ccf_overlay.mat')['allen_ccf']

In [5]:
from numpy.random import seed
seed(1)
keras.utils.set_random_seed(1)

## Analysis

In [6]:
## Set modeling parameters

subjects = [4,5,6,7,8,9,10]
delay = 1 # implement delay embedding? 0 or 1
nonlinear = 1 # use nonlinear mappings (VAE) (as opposed to linear regression) ? 0 or 1
Fs = 20 # sampling frequency
dt = 1/Fs
stackmax = 100 # Hankel row dimension (i.e., number of time shifts)
spacing = 3 # number of samples separating each row
r = 10 # number of modes of the Hankel matrix to keep (used for projection onto top r Legendre polynomials)
z_n = 4 # number of latent dimensions used for VAE
num_epochs = 200 # number of training epochs
lr = .001 # learning rate
beta = .1 # beta weight for KL loss in VAE training
anneal_step = 2000 # determines rate at which KL loss is increased during training
pretrained_enc = [] # pretrained encoder

In [7]:
## Create group files

xtrains = []
pupils = []
ytrains = []
yscales = []
lags = []

modality='jrgeco'

for s in range(len(subjects)):
    
    tic()
    
    subj = subjects[s]
    
    # Load data
    pupil = load_pupil(subj)
    brain = load_brain(subj,modality)
    
    # Compute lags for later
    lag_range = 5 # range over which to look for optimal lag between pupil and brain cross-correlation (in seconds)
    lag_range = int(lag_range/dt)
    lag,_,_ = find_opt_lag(pupil,np.mean(brain,axis=1),lag_range)
    lags.append(lag)
          
    ## Time delay embedding
    polys = legendre_polys(r, stackmax)
    Hp = hankel_matrix(pupil.T,stackmax,spacing)
    xtrain = Hp.T@polys
    xtrain = np.concatenate((xtrain,Hp.T[:,-1:]),axis=1)
    ytrain = brain[spacing*(stackmax-1):,:]
        
    xtrain,_ = scaler(xtrain)
    ytrain, scale_y = scaler(ytrain)
    
    xtrains.append(xtrain)
    ytrains.append(ytrain)
    yscales.append(scale_y)
    
    toc()


Elapsed time is 8.31776738166809 seconds.
Elapsed time is 8.247560501098633 seconds.
Elapsed time is 8.162980079650879 seconds.
Elapsed time is 8.19522738456726 seconds.
Elapsed time is 8.287102937698364 seconds.
Elapsed time is 8.337523460388184 seconds.
Elapsed time is 8.114526748657227 seconds.


In [8]:
np.save(outdir + '/group_' + modality + '_lags.npy', lags)
#np.save(outdir + '/group_' + modality + '_ytrains.npy', ytrains, allow_pickle=True)
#np.save(outdir + '/group_xtrains.npy', xtrains)

In [9]:
# Leave-one-out crossval

num_PCs = 10
modality = 'jrgeco'

#ytrains = np.load(outdir + '/group_' + modality + '_ytrains.npy',allow_pickle=True)
#lags = np.load(outdir + '/group_' + modality + '_lags.npy').tolist()

for s in range(len(xtrains)):
    
    tic()

    print(f'Fold {s+1}')
    
    subj = subjects[s]
        
    for m in range(4):
        if m==0:
            delay = 1
            nonlinear = 1
        elif m==1:
            delay = 1
            nonlinear = 0
        elif m==2:
            delay = 0
            nonlinear = 1
        else:
            delay = 0
            nonlinear = 0
           
        # Apply the median lag adjustment to all mice
        lag = int(np.median(lags[:s] + lags[s+1:]))
        
        xtrains_aligned = xtrains.copy()
        ytrains_aligned = ytrains.copy()
        
        for ss in range(len(xtrains)):
            xtrains_aligned[ss] = xtrains_aligned[ss][lag:,:]
            ytrains_aligned[ss] = ytrains_aligned[ss][:-lag,:]
        
        train_x = np.concatenate(xtrains_aligned[:s] + xtrains_aligned[s+1:], axis=0)
        train_y = np.concatenate(ytrains_aligned[:s] + ytrains_aligned[s+1:], axis=0)

        test_x = xtrains_aligned[s]
        
        # Time delay embedding    
        if not delay:
            train_x = train_x[:,-1:]
            test_x = test_x[:,-1:]

        # Project to top group PCs for efficient training
        u,sigma,vh = rsvd(train_y, n_components=num_PCs)

        train_y = train_y@vh.T[:,:num_PCs]

        # Preprocessing
        train_x_sc, scale_x = scaler(train_x)
        train_y_sc, scale_y = scaler(train_y)
        
        # Train model
        if nonlinear:
            r_squared, model, encoder, decoder = VAE(train_x_sc, train_y_sc, latent_dim=z_n, beta=beta,
                                                     num_epochs=num_epochs, anneal_step=anneal_step, lr=lr)
        else:
            r_squared, model = linear_regression(train_x_sc, train_y_sc)

        # Test model    
        test_x_sc,_ = scaler(test_x, scale_x)
        
        test_y_hat_sc = model.predict(test_x_sc)
        test_y_hat = scale_y.inverse_transform(test_y_hat_sc)        
        test_y_hat = yscales[s].inverse_transform(test_y_hat@vh[:num_PCs,:]) # project back to brain space
        
        test_y = load_brain(subj,modality)[spacing*(stackmax-1):,:][:-lag,:]

        # Save files

        if delay & nonlinear:
            np.save(outdir + '/' + str(subj) + '_' + modality + '_delay_xtest.npy', test_x_sc)
            np.save(outdir + '/' + str(subj) + '_' + modality + '_ytest.npy', test_y)
            np.save(outdir + '/' + str(subj) + '_' + modality + '_delay_nonlin_ytest_hat.npy', test_y_hat)
            
            encoder.save_weights(outdir + '/' + str(subj) + '_' + modality + '_encoder_weights.h5')
            decoder.save_weights(outdir + '/' + str(subj) + '_' + modality + '_decoder_weights.h5')

        elif delay:
            np.save(outdir + '/' + str(subj) + '_' + modality + '_delay_lin_ytest_hat.npy', test_y_hat)

        elif nonlinear:
            np.save(outdir + '/' + str(subj) + '_' + modality + '_nodelay_nonlin_ytest_hat.npy', test_y_hat)
        
        else:
            np.save(outdir + '/' + str(subj) + '_' + modality + '_nodelay_lin_ytest_hat.npy', test_y_hat)
        
    toc()

Fold 1


2025-01-02 20:05:26.532328: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-02 20:05:28.431443: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9627 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:19:00.0, compute capability: 7.5
2025-01-02 20:05:28.432227: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9627 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5
2025-01-02 20:05:28.432821: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/repli

Epoch 1/200
60/60 [==============================] - 1s 5ms/step - loss: 1.1150 - reconstruction_loss: 1.0781 - kl_loss: 2.5847e-04 - kl_anneal: 0.0015
Epoch 2/200
60/60 [==============================] - 0s 6ms/step - loss: 1.0078 - reconstruction_loss: 0.9992 - kl_loss: 6.6012e-04 - kl_anneal: 0.0045
Epoch 3/200
60/60 [==============================] - 0s 6ms/step - loss: 0.9666 - reconstruction_loss: 0.9565 - kl_loss: 0.0010 - kl_anneal: 0.0075
Epoch 4/200
60/60 [==============================] - 0s 6ms/step - loss: 0.9309 - reconstruction_loss: 0.9192 - kl_loss: 0.0014 - kl_anneal: 0.0105
Epoch 5/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8976 - reconstruction_loss: 0.8870 - kl_loss: 0.0020 - kl_anneal: 0.0135
Epoch 6/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8734 - reconstruction_loss: 0.8645 - kl_loss: 0.0026 - kl_anneal: 0.0165
Epoch 7/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8527 - reconstruction_loss: 0

Epoch 56/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7898 - reconstruction_loss: 0.7825 - kl_loss: 0.0110 - kl_anneal: 0.1000
Epoch 57/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7932 - reconstruction_loss: 0.7827 - kl_loss: 0.0110 - kl_anneal: 0.1000
Epoch 58/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7914 - reconstruction_loss: 0.7823 - kl_loss: 0.0110 - kl_anneal: 0.1000
Epoch 59/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7896 - reconstruction_loss: 0.7818 - kl_loss: 0.0111 - kl_anneal: 0.1000
Epoch 60/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7934 - reconstruction_loss: 0.7820 - kl_loss: 0.0110 - kl_anneal: 0.1000
Epoch 61/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7984 - reconstruction_loss: 0.7812 - kl_loss: 0.0110 - kl_anneal: 0.1000
Epoch 62/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7953 - reconstruction_loss: 0.

60/60 [==============================] - 0s 6ms/step - loss: 0.7828 - reconstruction_loss: 0.7739 - kl_loss: 0.0105 - kl_anneal: 0.1000
Epoch 111/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7888 - reconstruction_loss: 0.7731 - kl_loss: 0.0104 - kl_anneal: 0.1000
Epoch 112/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7859 - reconstruction_loss: 0.7732 - kl_loss: 0.0104 - kl_anneal: 0.1000
Epoch 113/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7833 - reconstruction_loss: 0.7745 - kl_loss: 0.0105 - kl_anneal: 0.1000
Epoch 114/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7827 - reconstruction_loss: 0.7742 - kl_loss: 0.0105 - kl_anneal: 0.1000
Epoch 115/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7818 - reconstruction_loss: 0.7728 - kl_loss: 0.0105 - kl_anneal: 0.1000
Epoch 116/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7822 - reconstruction_loss: 0.7727 - 

...

60/60 [==============================] - 0s 6ms/step - loss: 0.7765 - reconstruction_loss: 0.7646 - kl_loss: 0.0116 - kl_anneal: 0.1000
Epoch 155/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7765 - reconstruction_loss: 0.7647 - kl_loss: 0.0116 - kl_anneal: 0.1000
Epoch 156/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7732 - reconstruction_loss: 0.7639 - kl_loss: 0.0116 - kl_anneal: 0.1000
Epoch 157/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7780 - reconstruction_loss: 0.7637 - kl_loss: 0.0116 - kl_anneal: 0.1000
Epoch 158/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7737 - reconstruction_loss: 0.7646 - kl_loss: 0.0116 - kl_anneal: 0.1000
Epoch 159/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7764 - reconstruction_loss: 0.7638 - kl_loss: 0.0116 - kl_anneal: 0.1000
Epoch 160/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7769 - reconstruction_loss: 0.7642 - 

60/60 [==============================] - 0s 5ms/step - loss: 0.9141 - reconstruction_loss: 0.9044 - kl_loss: 0.0037 - kl_anneal: 0.0225
Epoch 9/200
60/60 [==============================] - 0s 6ms/step - loss: 0.9047 - reconstruction_loss: 0.9007 - kl_loss: 0.0039 - kl_anneal: 0.0255
Epoch 10/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9014 - reconstruction_loss: 0.8968 - kl_loss: 0.0042 - kl_anneal: 0.0285
Epoch 11/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9027 - reconstruction_loss: 0.8936 - kl_loss: 0.0045 - kl_anneal: 0.0315
Epoch 12/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8975 - reconstruction_loss: 0.8932 - kl_loss: 0.0047 - kl_anneal: 0.0345
Epoch 13/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8935 - reconstruction_loss: 0.8921 - kl_loss: 0.0048 - kl_anneal: 0.0375
Epoch 14/200
60/60 [==============================] - 0s 6ms/step - loss: 0.8954 - reconstruction_loss: 0.8907 - kl_loss

60/60 [==============================] - 0s 6ms/step - loss: 0.8911 - reconstruction_loss: 0.8883 - kl_loss: 0.0048 - kl_anneal: 0.1000
Epoch 64/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8936 - reconstruction_loss: 0.8887 - kl_loss: 0.0048 - kl_anneal: 0.1000
Epoch 65/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8947 - reconstruction_loss: 0.8880 - kl_loss: 0.0048 - kl_anneal: 0.1000
Epoch 66/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8927 - reconstruction_loss: 0.8882 - kl_loss: 0.0048 - kl_anneal: 0.1000
Epoch 67/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8914 - reconstruction_loss: 0.8888 - kl_loss: 0.0048 - kl_anneal: 0.1000
Epoch 68/200
60/60 [==============================] - 0s 6ms/step - loss: 0.8902 - reconstruction_loss: 0.8878 - kl_loss: 0.0048 - kl_anneal: 0.1000
Epoch 69/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8913 - reconstruction_loss: 0.8877 - kl_los

60/60 [==============================] - 0s 5ms/step - loss: 0.8922 - reconstruction_loss: 0.8860 - kl_loss: 0.0041 - kl_anneal: 0.1000
Epoch 118/200
60/60 [==============================] - 0s 6ms/step - loss: 0.8903 - reconstruction_loss: 0.8856 - kl_loss: 0.0040 - kl_anneal: 0.1000
Epoch 119/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8875 - reconstruction_loss: 0.8856 - kl_loss: 0.0040 - kl_anneal: 0.1000
Epoch 120/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8899 - reconstruction_loss: 0.8858 - kl_loss: 0.0040 - kl_anneal: 0.1000
Epoch 121/200
60/60 [==============================] - 0s 6ms/step - loss: 0.8889 - reconstruction_loss: 0.8856 - kl_loss: 0.0040 - kl_anneal: 0.1000
Epoch 122/200
60/60 [==============================] - 0s 6ms/step - loss: 0.8910 - reconstruction_loss: 0.8864 - kl_loss: 0.0040 - kl_anneal: 0.1000
Epoch 123/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8909 - reconstruction_loss: 0.8861 - 

60/60 [==============================] - 0s 6ms/step - loss: 0.8861 - reconstruction_loss: 0.8847 - kl_loss: 0.0039 - kl_anneal: 0.1000
Epoch 172/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8897 - reconstruction_loss: 0.8846 - kl_loss: 0.0039 - kl_anneal: 0.1000
Epoch 173/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8935 - reconstruction_loss: 0.8847 - kl_loss: 0.0039 - kl_anneal: 0.1000
Epoch 174/200
60/60 [==============================] - 0s 6ms/step - loss: 0.8896 - reconstruction_loss: 0.8845 - kl_loss: 0.0039 - kl_anneal: 0.1000
Epoch 175/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8888 - reconstruction_loss: 0.8842 - kl_loss: 0.0039 - kl_anneal: 0.1000
Epoch 176/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8900 - reconstruction_loss: 0.8845 - kl_loss: 0.0039 - kl_anneal: 0.1000
Epoch 177/200
60/60 [==============================] - 0s 6ms/step - loss: 0.8944 - reconstruction_loss: 0.8845 - 

# FAD

In [10]:
## Create group files
xtrains = []
pupils = []
ytrains = []
yscales = []
lags = []

modality='fad'

for s in range(len(subjects)):
    
    tic()
    
    subj = subjects[s]
    
    # Load data
    pupil = load_pupil(subj)
    brain = load_brain(subj,modality)
    
    # Compute lags for later
    lag_range = 5 # range over which to look for optimal lag between pupil and brain cross-correlation (in seconds)
    lag_range = int(lag_range/dt)
    lag,_,_ = find_opt_lag(pupil,np.mean(brain,axis=1),lag_range)
    lags.append(lag)
          
    ## Time delay embedding
    polys = legendre_polys(r, stackmax)
    Hp = hankel_matrix(pupil.T,stackmax,spacing)
    xtrain = Hp.T@polys
    xtrain = np.concatenate((xtrain,Hp.T[:,-1:]),axis=1)
    ytrain = brain[spacing*(stackmax-1):,:]
        
    xtrain,_ = scaler(xtrain)
    ytrain, scale_y = scaler(ytrain)
    
    xtrains.append(xtrain)
    ytrains.append(ytrain)
    yscales.append(scale_y)
    
    toc()


Elapsed time is 8.219435930252075 seconds.
Elapsed time is 8.585155248641968 seconds.
Elapsed time is 8.518546342849731 seconds.
Elapsed time is 8.57174277305603 seconds.
Elapsed time is 8.18059778213501 seconds.
Elapsed time is 8.576695919036865 seconds.
Elapsed time is 8.902997255325317 seconds.


In [11]:
# Leave-one-out crossval

num_PCs = 10
modality = 'fad'

#ytrains = np.load(outdir + '/group_' + modality + '_ytrains.npy',allow_pickle=True)
#lags = np.load(outdir + '/group_' + modality + '_lags.npy').tolist()

for s in range(len(xtrains)):
    
    tic()

    print(f'Fold {s+1}')
    
    subj = subjects[s]
        
    for m in range(4):
        if m==0:
            delay = 1
            nonlinear = 1
        elif m==1:
            delay = 1
            nonlinear = 0
        elif m==2:
            delay = 0
            nonlinear = 1
        else:
            delay = 0
            nonlinear = 0
           
        # Apply the median lag adjustment to all mice
        lag = int(np.median(lags[:s] + lags[s+1:]))
        
        xtrains_aligned = xtrains.copy()
        ytrains_aligned = ytrains.copy()
        
        for ss in range(len(xtrains)):
            xtrains_aligned[ss] = xtrains_aligned[ss][lag:,:]
            ytrains_aligned[ss] = ytrains_aligned[ss][:-lag,:]
        
        train_x = np.concatenate(xtrains_aligned[:s] + xtrains_aligned[s+1:], axis=0)
        train_y = np.concatenate(ytrains_aligned[:s] + ytrains_aligned[s+1:], axis=0)

        test_x = xtrains_aligned[s]
        
        # Time delay embedding    
        if not delay:
            train_x = train_x[:,-1:]
            test_x = test_x[:,-1:]

        # Project to top group PCs for efficient training
        u,sigma,vh = rsvd(train_y, n_components=num_PCs)

        train_y = train_y@vh.T[:,:num_PCs]

        # Preprocessing
        train_x_sc, scale_x = scaler(train_x)
        train_y_sc, scale_y = scaler(train_y)
        
        # Train model
        if nonlinear:
            r_squared, model, encoder, decoder = VAE(train_x_sc, train_y_sc, latent_dim=z_n, beta=beta,
                                                     num_epochs=num_epochs, anneal_step=anneal_step, lr=lr)
        else:
            r_squared, model = linear_regression(train_x_sc, train_y_sc)

        # Test model    
        test_x_sc,_ = scaler(test_x, scale_x)
        
        test_y_hat_sc = model.predict(test_x_sc)
        test_y_hat = scale_y.inverse_transform(test_y_hat_sc)
        test_y_hat = yscales[s].inverse_transform(test_y_hat@vh[:num_PCs,:]) # project back to brain space
        
        test_y = load_brain(subj,modality)[spacing*(stackmax-1):,:][:-lag,:]

        # Save files

        if delay & nonlinear:
            np.save(outdir + '/' + str(subj) + '_' + modality + '_delay_xtest.npy', test_x_sc)
            np.save(outdir + '/' + str(subj) + '_' + modality + '_ytest.npy', test_y)
            np.save(outdir + '/' + str(subj) + '_' + modality + '_delay_nonlin_ytest_hat.npy', test_y_hat)
            
            #encoder.save_weights(outdir + '/' + str(subj) + '_' + modality + '_encoder_weights.h5')
            #decoder.save_weights(outdir + '/' + str(subj) + '_' + modality + '_decoder_weights.h5')

        elif delay:
            np.save(outdir + '/' + str(subj) + '_' + modality + '_delay_lin_ytest_hat.npy', test_y_hat)

        elif nonlinear:
            np.save(outdir + '/' + str(subj) + '_' + modality + '_nodelay_nonlin_ytest_hat.npy', test_y_hat)
        
        else:
            np.save(outdir + '/' + str(subj) + '_' + modality + '_nodelay_lin_ytest_hat.npy', test_y_hat)
        
    toc()

Fold 1
Epoch 1/200
60/60 [==============================] - 1s 5ms/step - loss: 1.0315 - reconstruction_loss: 1.0135 - kl_loss: 2.1488e-04 - kl_anneal: 0.0015
Epoch 2/200
60/60 [==============================] - 0s 6ms/step - loss: 0.9790 - reconstruction_loss: 0.9685 - kl_loss: 5.8120e-04 - kl_anneal: 0.0045
Epoch 3/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9324 - reconstruction_loss: 0.9205 - kl_loss: 9.7865e-04 - kl_anneal: 0.0075
Epoch 4/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8877 - reconstruction_loss: 0.8728 - kl_loss: 0.0015 - kl_anneal: 0.0105
Epoch 5/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8418 - reconstruction_loss: 0.8386 - kl_loss: 0.0021 - kl_anneal: 0.0135
Epoch 6/200
60/60 [==============================] - 0s 6ms/step - loss: 0.8281 - reconstruction_loss: 0.8233 - kl_loss: 0.0028 - kl_anneal: 0.0165
Epoch 7/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8223 - reconstruct

60/60 [==============================] - 0s 5ms/step - loss: 0.7781 - reconstruction_loss: 0.7691 - kl_loss: 0.0116 - kl_anneal: 0.1000
Epoch 56/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7803 - reconstruction_loss: 0.7693 - kl_loss: 0.0117 - kl_anneal: 0.1000
Epoch 57/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7819 - reconstruction_loss: 0.7691 - kl_loss: 0.0117 - kl_anneal: 0.1000
Epoch 58/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7825 - reconstruction_loss: 0.7693 - kl_loss: 0.0117 - kl_anneal: 0.1000
Epoch 59/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7755 - reconstruction_loss: 0.7684 - kl_loss: 0.0117 - kl_anneal: 0.1000
Epoch 60/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7784 - reconstruction_loss: 0.7678 - kl_loss: 0.0117 - kl_anneal: 0.1000
Epoch 61/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7777 - reconstruction_loss: 0.7680 - kl_los

60/60 [==============================] - 0s 5ms/step - loss: 0.7725 - reconstruction_loss: 0.7594 - kl_loss: 0.0121 - kl_anneal: 0.1000
Epoch 111/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7722 - reconstruction_loss: 0.7588 - kl_loss: 0.0120 - kl_anneal: 0.1000
Epoch 112/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7676 - reconstruction_loss: 0.7577 - kl_loss: 0.0120 - kl_anneal: 0.1000
Epoch 113/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7706 - reconstruction_loss: 0.7588 - kl_loss: 0.0121 - kl_anneal: 0.1000
Epoch 114/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7670 - reconstruction_loss: 0.7577 - kl_loss: 0.0120 - kl_anneal: 0.1000
Epoch 115/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7728 - reconstruction_loss: 0.7581 - kl_loss: 0.0120 - kl_anneal: 0.1000
Epoch 116/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7712 - reconstruction_loss: 0.7573 - 

60/60 [==============================] - 0s 5ms/step - loss: 0.7617 - reconstruction_loss: 0.7506 - kl_loss: 0.0123 - kl_anneal: 0.1000
Epoch 165/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7594 - reconstruction_loss: 0.7508 - kl_loss: 0.0123 - kl_anneal: 0.1000
Epoch 166/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7619 - reconstruction_loss: 0.7509 - kl_loss: 0.0123 - kl_anneal: 0.1000
Epoch 167/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7611 - reconstruction_loss: 0.7515 - kl_loss: 0.0124 - kl_anneal: 0.1000
Epoch 168/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7607 - reconstruction_loss: 0.7503 - kl_loss: 0.0123 - kl_anneal: 0.1000
Epoch 169/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7610 - reconstruction_loss: 0.7499 - kl_loss: 0.0122 - kl_anneal: 0.1000
Epoch 170/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7687 - reconstruction_loss: 0.7502 - 

60/60 [==============================] - 0s 5ms/step - loss: 0.9186 - reconstruction_loss: 0.9112 - kl_loss: 0.0055 - kl_anneal: 0.0525
Epoch 19/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9140 - reconstruction_loss: 0.9093 - kl_loss: 0.0055 - kl_anneal: 0.0555
Epoch 20/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9131 - reconstruction_loss: 0.9092 - kl_loss: 0.0055 - kl_anneal: 0.0585
Epoch 21/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9167 - reconstruction_loss: 0.9104 - kl_loss: 0.0054 - kl_anneal: 0.0615
Epoch 22/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9168 - reconstruction_loss: 0.9085 - kl_loss: 0.0053 - kl_anneal: 0.0645
Epoch 23/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9131 - reconstruction_loss: 0.9094 - kl_loss: 0.0052 - kl_anneal: 0.0675
Epoch 24/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9140 - reconstruction_loss: 0.9097 - kl_los

...

60/60 [==============================] - 0s 5ms/step - loss: 0.7638 - reconstruction_loss: 0.7549 - kl_loss: 0.0112 - kl_anneal: 0.1000
Epoch 155/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7726 - reconstruction_loss: 0.7571 - kl_loss: 0.0113 - kl_anneal: 0.1000
Epoch 156/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7623 - reconstruction_loss: 0.7564 - kl_loss: 0.0113 - kl_anneal: 0.1000
Epoch 157/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7681 - reconstruction_loss: 0.7571 - kl_loss: 0.0113 - kl_anneal: 0.1000
Epoch 158/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7674 - reconstruction_loss: 0.7569 - kl_loss: 0.0113 - kl_anneal: 0.1000
Epoch 159/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7657 - reconstruction_loss: 0.7565 - kl_loss: 0.0113 - kl_anneal: 0.1000
Epoch 160/200
60/60 [==============================] - 0s 5ms/step - loss: 0.7684 - reconstruction_loss: 0.7554 - 

60/60 [==============================] - 0s 6ms/step - loss: 0.9219 - reconstruction_loss: 0.9223 - kl_loss: 0.0020 - kl_anneal: 0.0225
Epoch 9/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9222 - reconstruction_loss: 0.9209 - kl_loss: 0.0022 - kl_anneal: 0.0255
Epoch 10/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9169 - reconstruction_loss: 0.9191 - kl_loss: 0.0024 - kl_anneal: 0.0285
Epoch 11/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9181 - reconstruction_loss: 0.9184 - kl_loss: 0.0026 - kl_anneal: 0.0315
Epoch 12/200
60/60 [==============================] - 0s 6ms/step - loss: 0.9179 - reconstruction_loss: 0.9173 - kl_loss: 0.0028 - kl_anneal: 0.0345
Epoch 13/200
60/60 [==============================] - 0s 6ms/step - loss: 0.9158 - reconstruction_loss: 0.9159 - kl_loss: 0.0030 - kl_anneal: 0.0375
Epoch 14/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9233 - reconstruction_loss: 0.9146 - kl_loss

60/60 [==============================] - 0s 5ms/step - loss: 0.9119 - reconstruction_loss: 0.9040 - kl_loss: 0.0043 - kl_anneal: 0.1000
Epoch 64/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9076 - reconstruction_loss: 0.9046 - kl_loss: 0.0043 - kl_anneal: 0.1000
Epoch 65/200
60/60 [==============================] - 0s 6ms/step - loss: 0.9050 - reconstruction_loss: 0.9055 - kl_loss: 0.0043 - kl_anneal: 0.1000
Epoch 66/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9049 - reconstruction_loss: 0.9051 - kl_loss: 0.0043 - kl_anneal: 0.1000
Epoch 67/200
60/60 [==============================] - 0s 6ms/step - loss: 0.9097 - reconstruction_loss: 0.9044 - kl_loss: 0.0042 - kl_anneal: 0.1000
Epoch 68/200
60/60 [==============================] - 0s 6ms/step - loss: 0.9109 - reconstruction_loss: 0.9037 - kl_loss: 0.0043 - kl_anneal: 0.1000
Epoch 69/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9078 - reconstruction_loss: 0.9037 - kl_los

60/60 [==============================] - 0s 5ms/step - loss: 0.9073 - reconstruction_loss: 0.9006 - kl_loss: 0.0035 - kl_anneal: 0.1000
Epoch 118/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9070 - reconstruction_loss: 0.9021 - kl_loss: 0.0035 - kl_anneal: 0.1000
Epoch 119/200
60/60 [==============================] - 0s 6ms/step - loss: 0.9051 - reconstruction_loss: 0.9019 - kl_loss: 0.0035 - kl_anneal: 0.1000
Epoch 120/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9044 - reconstruction_loss: 0.9008 - kl_loss: 0.0035 - kl_anneal: 0.1000
Epoch 121/200
60/60 [==============================] - 0s 6ms/step - loss: 0.9059 - reconstruction_loss: 0.9019 - kl_loss: 0.0035 - kl_anneal: 0.1000
Epoch 122/200
60/60 [==============================] - 0s 6ms/step - loss: 0.9075 - reconstruction_loss: 0.9017 - kl_loss: 0.0035 - kl_anneal: 0.1000
Epoch 123/200
60/60 [==============================] - 0s 5ms/step - loss: 0.8995 - reconstruction_loss: 0.9029 - 

60/60 [==============================] - 0s 5ms/step - loss: 0.9032 - reconstruction_loss: 0.9015 - kl_loss: 0.0033 - kl_anneal: 0.1000
Epoch 172/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9057 - reconstruction_loss: 0.9007 - kl_loss: 0.0033 - kl_anneal: 0.1000
Epoch 173/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9081 - reconstruction_loss: 0.9019 - kl_loss: 0.0032 - kl_anneal: 0.1000
Epoch 174/200
60/60 [==============================] - 0s 6ms/step - loss: 0.9054 - reconstruction_loss: 0.9016 - kl_loss: 0.0033 - kl_anneal: 0.1000
Epoch 175/200
60/60 [==============================] - 0s 6ms/step - loss: 0.9025 - reconstruction_loss: 0.9023 - kl_loss: 0.0033 - kl_anneal: 0.1000
Epoch 176/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9016 - reconstruction_loss: 0.9020 - kl_loss: 0.0033 - kl_anneal: 0.1000
Epoch 177/200
60/60 [==============================] - 0s 5ms/step - loss: 0.9029 - reconstruction_loss: 0.9011 - 

## Hemo

In [12]:
## Create group files
xtrains = []
pupils = []
ytrains = []
yscales = []
lags = []

modality='hemo1'

for s in range(len(subjects)):
    
    tic()
    
    subj = subjects[s]
    
    # Load data
    pupil = load_pupil(subj)
    brain = load_brain(subj,modality)
    
    # Compute lags for later
    lag_range = 5 # range over which to look for optimal lag between pupil and brain cross-correlation (in seconds)
    lag_range = int(lag_range/dt)
    lag,_,_ = find_opt_lag(pupil,np.mean(brain,axis=1),lag_range)
    lags.append(lag)
          
    ## Time delay embedding
    polys = legendre_polys(r, stackmax)
    Hp = hankel_matrix(pupil.T,stackmax,spacing)
    xtrain = Hp.T@polys
    xtrain = np.concatenate((xtrain,Hp.T[:,-1:]),axis=1)
    ytrain = brain[spacing*(stackmax-1):,:]
        
    xtrain,_ = scaler(xtrain)
    ytrain, scale_y = scaler(ytrain)
    
    xtrains.append(xtrain)
    ytrains.append(ytrain)
    yscales.append(scale_y)
    
    toc()


Elapsed time is 8.591625690460205 seconds.
Elapsed time is 8.812855005264282 seconds.
Elapsed time is 8.293935775756836 seconds.
Elapsed time is 10.128209114074707 seconds.
Elapsed time is 9.665421724319458 seconds.
Elapsed time is 10.353856086730957 seconds.
Elapsed time is 10.097635269165039 seconds.


In [13]:
# Leave-one-out crossval

num_PCs = 10

#ytrains = np.load(outdir + '/group_' + modality + '_ytrains.npy',allow_pickle=True)
#lags = np.load(outdir + '/group_' + modality + '_lags.npy').tolist()

for s in range(len(xtrains)):
    
    tic()

    print(f'Fold {s+1}')
    
    subj = subjects[s]
        
    for m in range(4):
        if m==0:
            delay = 1
            nonlinear = 1
        elif m==1:
            delay = 1
            nonlinear = 0
        elif m==2:
            delay = 0
            nonlinear = 1
        else:
            delay = 0
            nonlinear = 0
           
        # Apply the median lag adjustment to all mice
        lag = int(np.median(lags[:s] + lags[s+1:]))
        
        xtrains_aligned = xtrains.copy()
        ytrains_aligned = ytrains.copy()
        
        for ss in range(len(xtrains)):
            xtrains_aligned[ss] = xtrains_aligned[ss][lag:,:]
            ytrains_aligned[ss] = ytrains_aligned[ss][:-lag,:]
        
        train_x = np.concatenate(xtrains_aligned[:s] + xtrains_aligned[s+1:], axis=0)
        train_y = np.concatenate(ytrains_aligned[:s] + ytrains_aligned[s+1:], axis=0)

        test_x = xtrains_aligned[s]
        
        # Time delay embedding    
        if not delay:
            train_x = train_x[:,-1:]
            test_x = test_x[:,-1:]

        # Project to top group PCs for efficient training
        u,sigma,vh = rsvd(train_y, n_components=num_PCs)

        train_y = train_y@vh.T[:,:num_PCs]

        # Preprocessing
        train_x_sc, scale_x = scaler(train_x)
        train_y_sc, scale_y = scaler(train_y)
        
        # Train model
        if nonlinear:
            r_squared, model, encoder, decoder = VAE(train_x_sc, train_y_sc, latent_dim=z_n, beta=beta,
                                                num_epochs=num_epochs, anneal_step=anneal_step, lr=lr, verbose=False)
        else:
            r_squared, model = linear_regression(train_x_sc, train_y_sc)

        # Test model    
        test_x_sc,_ = scaler(test_x, scale_x)
        
        test_y_hat_sc = model.predict(test_x_sc)
        test_y_hat = scale_y.inverse_transform(test_y_hat_sc)        
        test_y_hat = yscales[s].inverse_transform(test_y_hat@vh[:num_PCs,:]) # project back to brain space
        
        test_y = load_brain(subj,modality)[spacing*(stackmax-1):,:][:-lag,:]

        # Save files

        if delay & nonlinear:
            np.save(outdir + '/' + str(subj) + '_' + modality + '_delay_xtest.npy', test_x_sc)
            np.save(outdir + '/' + str(subj) + '_' + modality + '_ytest.npy', test_y)
            np.save(outdir + '/' + str(subj) + '_' + modality + '_delay_nonlin_ytest_hat.npy', test_y_hat)
            
            #encoder.save_weights(outdir + '/' + str(subj) + '_' + modality + '_encoder_weights.h5')
            #decoder.save_weights(outdir + '/' + str(subj) + '_' + modality + '_decoder_weights.h5')

        elif delay:
            np.save(outdir + '/' + str(subj) + '_' + modality + '_delay_lin_ytest_hat.npy', test_y_hat)

        elif nonlinear:
            np.save(outdir + '/' + str(subj) + '_' + modality + '_nodelay_nonlin_ytest_hat.npy', test_y_hat)
        
        else:
            np.save(outdir + '/' + str(subj) + '_' + modality + '_nodelay_lin_ytest_hat.npy', test_y_hat)
        
    toc()

Fold 1
Elapsed time is 187.22393536567688 seconds.
Fold 2
Elapsed time is 188.78913855552673 seconds.
Fold 3
Elapsed time is 187.02867794036865 seconds.
Fold 4
Elapsed time is 187.7639832496643 seconds.
Fold 5
Elapsed time is 186.94448685646057 seconds.
Fold 6
Elapsed time is 188.33775925636292 seconds.
Fold 7
Elapsed time is 187.92762732505798 seconds.
